In [86]:
with open('suihanki_wakati.txt', mode='r', encoding='utf-8') as f:
    text = f.read()

In [89]:
# 登場した単語のリスト
words = list(set(text.split()))

In [87]:
# w_size: ウィンドウ幅
def cooccurrence(text, w_size=1):
    import numpy as np

    # i文目のj語目が格納された二次元配列sentence(i, j)
    sentence = text.split('\n')
    for i in range(len(sentence)):
        sentence[i] = sentence[i].split()

    # 登場した単語のリスト
    words = list(set(text.split()))

    # words数サイズのゼロ行列を作る
    matrix = np.zeros((len(words), len(words)))

    # number番目の単語がw
    for number, w in enumerate(words):
        for i in range(len(sentence)):
            for j in range(len(sentence[i])):
                # wとi文目j単語目とが合致したら
                if w == sentence[i][j]:
                    # 行列の、窓幅分前の単語のところを+1する
                    for d in range(1, w_size+1):
                        if j-d >= 0:
                            matrix[number][words.index(sentence[i][j-d])] += 1
                    # 行列の、窓幅分後の単語のところを+1する
                    for d in range(1, w_size+1):
                        if j+d < len(sentence[i]):
                            matrix[number][words.index(sentence[i][j+d])] += 1
    return matrix

In [88]:
matrix = cooccurrence(text, w_size=1)

In [ ]:
# ベクトルの0でない成分を返す
# wは単語ベクトル
def F(w):
    